# Offline Rendering to Image / Video

This notebook shows a images and videos / animations can be rendered.

In [1]:
from vape4d import render
from vape4d.utils import diverging_alpha
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import os

os.environ['RUST_LOG'] = 'vape4d=debug'
os.environ['RUST_BACKTRACE'] = '1'

In [2]:
def volume(t,N=8):
    """Generate a 3D volume for a time step t

    Args:
        t (float): time
        N (int, optional): Number of points in each dimension. Defaults to 8.

    Returns:
        np.ndarray: 3D array of floats
    """
    x = np.linspace(-1,1,N)
    y = np.linspace(-1,1,N)
    z = np.linspace(-1,1,N)
    X,Y,Z = np.meshgrid(x,y,z)
    return np.sin(t*2*np.pi*(X+Y+Z))


## Static Image

In [3]:
# the diverging_alpha function creates a colormap that is transparent at the ends
cmap = diverging_alpha(plt.get_cmap("magma"))

# volume of shape [T,W,H,D]
data = np.stack([volume(t,N=64) for t in np.linspace(0,1,32)])


img = render(
    data.astype(np.float32), # expects float32 
    cmap,
    0.9, # time step between (0,1)
    width=512,
    height=512,
    distance_scale=10,
    background=(0,0,0,0) # transparent background
)
Image.fromarray(img)

thread '<unnamed>' panicked at src/lib.rs:94:14:
called `Option::unwrap()` on a `None` value
stack backtrace:
   0: rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic
   3: core::option::unwrap_failed
   4: vape4d::WGPUContext::new::{{closure}}
   5: pollster::block_on
   6: vape4d::py::vape4d::render_video
   7: vape4d::py::vape4d::<impl vape4d::py::vape4d::render_video::MakeDef>::_PYO3_DEF::trampoline
   8: cfunction_vectorcall_FASTCALL_KEYWORDS
             at /usr/local/src/conda/python-3.11.8/Objects/methodobject.c:443:24
   9: _PyObject_VectorcallTstate
             at /usr/local/src/conda/python-3.11.8/Include/internal/pycore_call.h:92:11
  10: PyObject_Vectorcall
             at /usr/local/src/conda/python-3.11.8/Objects/call.c:299:12
  11: _PyEval_EvalFrameDefault
             at /usr/local/src/conda/python-3.11.8/Python/ceval.c:4769:23
  12: _PyEval_EvalFrame
             at /usr/local/src/conda/python-3.11.8/Include/internal/pycore_ceval.h:73:16


PanicException: called `Option::unwrap()` on a `None` value

## Video

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML
fps = 24
video_duration = 3
n_frames = fps * video_duration
video = render(
    data, 
    cmap,
    np.linspace(0,1,n_frames), # time step between (0,1)
    width=512,
    height=512,
    distance_scale=10,
    background=(0,0,0,0) # transparent background
)

frames = [] # for storing the generated images
fig = plt.figure()
plt.axis('off')
for i in range(len(video)):
    frames.append([plt.imshow(video[i],animated=True)])

ani = animation.ArtistAnimation(fig, frames, interval=1000/fps, blit=True)
plt.close()
HTML(ani.to_jshtml())